<img src="https://www.e4ds.com/news_photo/U77C53G6CP8ASEHUJ5B7.png">

# 인코더, 디코더, 그리고 인코더-디코더 구조

- 명시적 표현(Explicit representation): 사람이 읽거나 소프트웨어가 바로 이해할 수 있는 데이터 형태 (예: 문장, 이미지, 데이터 포인트 등)
- 암묵적 표현(Implicit representation): 모델이 학습 과정에서 자동으로 만들어내는, 목표를 위해 최적화된 내부 표현 (예: 임베딩, 중간 레이어의 출력 등)

## GPT(Generate Pre-train Transformer)

In [ ]:
from transformers import pipeline, set_seed 

generator = pipeline('text-generation', model = 'gpt2')

# max_length -> return되는 token의 개수 
generator("Hello world", max_length = 20, num_return_sequences = 5)

Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello world, all for a good ol' boys-who-come-up-in-town"},
 {'generated_text': 'Hello world!\n\nThe game, named The Way of the Sun after God, is an ancient'},
 {'generated_text': "Hello world, they always try to change it in different ways: for instance if they're not giving"},
 {'generated_text': 'Hello world is full of surprises — every once-in-a-lifetime opportunity for the media'},
 {'generated_text': 'Hello world, you are my best friend.\n\nI am not your god\n\nA god'}]

### 한 번에 모든 토큰 생성 

- 병렬로 토큰을 생성하므로 속도 면에서 빠르다.
- 중간 개입이 어려우며, 통계 기반 예측을 한 번에 적용 
- `max_new_tokens = 20`

In [ ]:
import torch 

print("GENERATING ALL AT ONCE:")
input_str = 'Hello world'

print(f"{(x := generator.preprocess(input_str))}") # encoding을 해서 model에 입력 
print(f"{(x := generator.forward(x, max_new_tokens=20))}")
print(f"{(x := generator.postprocess(x))}") # decoding 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GENERATING ALL AT ONCE:
{'input_ids': tensor([[15496,   995]]), 'attention_mask': tensor([[1, 1]]), 'prompt_text': 'Hello world'}
{'generated_sequence': tensor([[[15496,   995,     0,   770,   318,   534,  1110,  2474,   198,   198,
              1,  5195,    11,   523,   703,   750,   428,  1645,  1701,   198,
            198,    51]]]), 'input_ids': tensor([[15496,   995]]), 'prompt_text': 'Hello world'}
[{'generated_text': 'Hello world! This is your day!"\n\n"Why, so how did this happen?"\n\nT'}]


### 한 번에 한 토큰씩 생성
- 토큰마다 모델을 호출하므로 느림
- 메모리 사용량이 많음 (매번 전체 context 재입력)
- 매 토큰마다 제어 가능
```python
for i in range(20):
    x = generator.forward(x, max_new_tokens = 1)
```

In [ ]:
print("\nGENERATING ONE TOKEN AT A TIME (prep+forward+post):")
print(input_str := "Hello world", end="")
output_buffer = ""
for i in range(20):
    # 전치리와 forward, 그리고 decoding을 각 타임마다 한 번만 수행 
    x = generator.preprocess(input_str + output_buffer)
    x = generator.forward(x, max_new_tokens=1)
    x = generator.postprocess(x)
    # GPT-2는 decoder-only 모델이므로 매번 이전 토큰 전체를 다시 입력으로 받은 후 다음 토큰 생성
    # input_str + output_buffer 만큼 x에서 get해서 다음 토큰 생성에 사용 
    next_word = x[0].get("generated_text")[len(input_str + output_buffer):]
    output_buffer += next_word
    print(next_word.replace("\n", "\\n"), end="")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



GENERATING ONE TOKEN AT A TIME (prep+forward+post):
Hello world,

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

 we'll be going into detail by the next day. The rest of this episode is

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


 an extension

### GPT-2 모델을 사용하여 토큰을 하나씩 수동으로 생성하는 로우레벨 구현

- 저수준 로직을 직접 구현하여 세밀하게 제어 가능
- GPT-2 `transformer` 직접 호출 (앞서 얘기한 방법들은 Hugging face가 처리)
- 제어력 강함 

In [12]:
print("\n\nGENERATING ONE TOKEN AT A TIME (manually, greedily-sampled):")

# Transformer의 model_body, head, tokenizer.encode, tokenizer.decode를 직접 호출해서 사용 
model_body = generator.model.transformer.to('cuda') # transformer의 GPT-2 모델을 호출해서 사용 
model_head = generator.model.lm_head.to('cuda')
tknzr_encode = generator.tokenizer.encode
tknzr_decode = generator.tokenizer.decode

def compute_embed(token_id):
    device = model_body.wte.weight.device
    # GPT-2의 word toekn embedding layer(wte)로 token_id를 임베딩 벡터로 변환 
    return model_body.wte(torch.tensor([token_id], device = device)).view(1, -1, 768)

# PREFILL stage: Processing the initial input string
print(input_str := "<|endoftext|> Hello world", end="")
# inputs = {k: v.to('cuda') for k, v in }
embed_buffer = compute_embed(tknzr_encode(input_str))
attention_mask = torch.ones(embed_buffer.shape[:2], dtype=torch.long) # 1로 채우면서 모든 토큰이 유효함 표시 
past_key_values = None # past_key_value 초기화 

# PREFILL - running the model for the input string, getting kv cache and embeddings
prefill_output = model_body.forward(
    inputs_embeds=embed_buffer, # token_id 대신 미리 정의한 임베딩 벡터를 직접 넘김 
    attention_mask=attention_mask, # attention mask 
    past_key_values=past_key_values,
)
past_key_values = prefill_output.get("past_key_values") # 각 레이어의 K/V cache이며 다음 토큰 계산 시 재사용 
predicted_embed = prefill_output.get("last_hidden_state") # 각 토큰의 최종 출력 벡터 



GENERATING ONE TOKEN AT A TIME (manually, greedily-sampled):
<|endoftext|> Hello world

In [13]:
for i in range(100):
    predicted_probs = model_head(predicted_embed[:, -1, :]) # 가장 최근 토큰의 임베딩만 추출 
    predicted_token = torch.argmax(predicted_probs, dim=-1).item()
    print(tknzr_decode(predicted_token), end="") # 다음에 올 확률이 가장 큰 token을 decoding하여 확인 

    # Update attention mask and run model with past_key_values for next token
    decode_output = model_body.forward(
        inputs_embeds=compute_embed(predicted_token), 
        attention_mask=torch.ones([1,1], dtype=torch.long),
        past_key_values=past_key_values,
    )
    predicted_embed = decode_output.get("last_hidden_state")
    past_key_values = decode_output.get("past_key_values")

!

I'm a programmer and I'm a big fan of the Java programming language. I've been using Java since I was a kid and I've been using it for a long time. I've been using Java for a long time and I've been using it for a long time. I've been using Java for a long time and I've been using it for a long time. I've been using Java for a long time and I've been using it for a long time.

## T5(Text-to-Text Transfer Transformer)

T5는 Google에서 2019년에 발표한 자연어처리 모델로, 모든 NLP 작업을 **Text-to-Text** 문제로 통일해서 처리하는 프레임워크이다. 기존에는 분류, 번역 ,요약, 질의응답 등 NLP task마다 서로 다른 아키텍처나 출력 형식을 사용하였는데 이를 하나의 통합된 모델이 다양한 작업을 처리할 수 있게 만들었다.

- 두 개 이상의 시퀀스를 다루고, 이 시퀀스들의 길이가 다를 수 있을 때
    - 만약 입력과 출력 길이가 같거나 출력이 입력의 부분집합이라면 인코더만으로 충분하다.
- 출력 시퀀스를 점진적으로 생성해야 할 때
    - 출력이 고정되어 있거나 요약하는 수준이라면 인코더만으로 충분하다.
- 입력과 출력 시퀀스가 서로 다른 분포(형식, 목적, 데이터 유형 등)를 따를 때
    - 동일한 분포라면 같은 네트워크 경로를 사용하는 게 더 낫다.
- 경량화나 특정 작업에 특화된 모델이 필요할 때
    - 범용 모델이라면 디코더 경로를 활용하는 편이 나을 수도 있다.

In [14]:
from transformers import pipeline

translator = pipeline('translation_en_to_fr', model = 't5-base', device = 'cuda')
translator(["Hello World! How's it going?", "What's your name?"])

c:\Users\user\anaconda3\envs\torchenv\lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--t5-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For b

[{'translation_text': 'Bonjour Monde, comment se passe-t-il ?'},
 {'translation_text': 'Quel est votre nom?'}]

### Preprocessing & Postprocessing

In [15]:
text_en = "Hello World! How's it going?"
resp_fr = translator(text_en)
text_fr = resp_fr[0]['translation_text']

tknzr = translator.tokenizer
tokens_ins = [tknzr.decode(x) for x in tknzr.encode(text_en)]
tokens_in2 = [tknzr.decode(x) for x in translator.preprocess(text_en)['input_ids'][0]]
tokens_out = [tknzr.decode(x) for x in tknzr.encode(text_fr)]
print(f"Inputs Into Preprocessing: {' | '.join(tokens_ins)}")
print(f"Inputs Into Model Forward: {' | '.join(tokens_in2)}")
print(f"Output From Model Forward: {' | '.join(tokens_out)}")

Inputs Into Preprocessing: Hello | World | ! | How | ' | s | it | going | ? | </s>
Inputs Into Model Forward: translate | English | to | French | : | Hello | World | ! | How | ' | s | it | going | ? | </s>
Output From Model Forward: Bonjour | Monde | , | comment | se | passe | - | t | - | il |  | ? | </s>


In [16]:
import torch

def get_token_generator(pipeline, model=None, tokenizer=None, max_tokens=50):
    
    model = pipeline.model or model
    tknzr = pipeline.tokenizer or tokenizer
    # getattr: model 객체에 encoder라는 속성이 있으면 그 값을 반환하고, 없으면 None을 반환 
    encoder = getattr(model, "encoder", None)
    decoder = ( ## Non-exhaustive resolution
        getattr(model, "decoder", None) 
        or getattr(model, "model", None) 
        or getattr(model, "transformer", None)
    )
    lm_head = getattr(model, "lm_head", None)
    dev = decoder.device
    
    def token_generator(
        encoder_input: str = "",
        decoder_input: str = "",
        max_tokens: int = max_tokens
    ):  
        # encode(encoder_input)[:-1]은 eos_token을 제거하는 용도
        # boolean을 활용하여 encoder_input이 빈 문자열이면 [] * 0 -> 오류 처리
        encoder_input_idxs = tknzr.encode(encoder_input)[:-1] * bool(encoder_input)
        decoder_input_idxs = tknzr.encode(decoder_input)[:-1] * bool(decoder_input)
        decoder_inputs = {}

        # 만약 encoder가 존재하면 input_ids 키를 가진 딕셔너리 생성 
        if encoder:
            encoder_inputs = {"input_ids": torch.tensor([encoder_input_idxs], device=dev)}
            encoder_outputs = encoder(**encoder_inputs)
            # encoider의 출력 임베딩을 디코더에 넘겨줘야 디코더가 인코더의 정보를 참고해서 출력을 생성할 수 있음 
            decoder_inputs["encoder_hidden_states"] = encoder_outputs.last_hidden_state
        elif encoder_input_idxs:
            print("`encoder_input` specified despite no encoder being available. Ignoring")
            
        # pad_token_id가 존재하면 넣어주고 없다면 빈 리스트 return 
        buffer_token_idxs = [] if (tknzr.pad_token_id is None) else [tknzr.pad_token_id]
        buffer_token_idxs += decoder_input_idxs # encoder의 출력 상태 더해줌 
        buffer_token_str = ""
        max_length = len(buffer_token_idxs) + max_tokens # max_length는 max_tokens 만큼 추가해서 정의 
        while len(buffer_token_idxs) < max_length:
            
            decoder_inputs["input_ids"] = torch.tensor([buffer_token_idxs], device=dev).long()
            decoder_outputs = decoder(**decoder_inputs) # decoder는 지금까지 생성된 context를 바탕으로 다음 토큰 분포 예측
            model_outputs = lm_head(decoder_outputs.last_hidden_state) # 토큰별 logits값 연산 
        
            ## Get the most likely next token and add it to the buffer
            try: 
                sampled_token_idx = torch.argmax(model_outputs, -1)[0][-1].item()
            except: 
                break
            
            buffer_token_idxs += [sampled_token_idx] # 새로 예측한 token ID를 생성 버퍼에 추가 
            buffer_token_old = buffer_token_str # 현재까지 생성된 텍스트 
            buffer_token_str = tknzr.decode(buffer_token_idxs)
            buffer_token_new = buffer_token_str[len(buffer_token_old):] # 이번에 생성된 단어를 확인하기 위해 

            ## Yield (output while keeping spot in the generator call) next token.
            ## If it's end-of-string </s>, break the loop.
            if sampled_token_idx == tknzr.eos_token_id:
                break
            if buffer_token_new:
                yield buffer_token_new
    
    return token_generator

###############################################################################

streamer = get_token_generator(translator)
input_raw_str = "translate English to French: Hello World! How's it going?</s>"

for token in streamer(encoder_input = input_raw_str):
    print(token, end="|")

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


<pad> Bonjour| Monde|,| comment| se| passe|-|t|-|il| |?|

### Google의 `flan-t5-large` 실습

<img src="https://velog.velcdn.com/images/choonsik_mom/post/275d137e-3cf4-42e2-841d-e19ba26908ee/image.png">

<img src="https://blog.kakaocdn.net/dna/8lWFY/btsngIa3NBA/AAAAAAAAAAAAAAAAAAAAAHSgJ1PhqhY8QqG2Tyw9HqvCob1YB6YuJF3vc4FvvkPU/img.png?credential=yqXZFxpELC7KVnFOS48ylbz2pIh7yKj8&expires=1753973999&allow_ip=&allow_referer=&signature=aQn90Gr909Y4AmplK8KYpkmUUWE%3D">

### Flan-T5 (Unsupervised learning)

Flan version의 T5 모델은 더욱 복잡한 작업들을 통해 추가로 학습되어, 기존에 학습한 목적을 넘어서 `in-context leraning` 능력을 갖출 수 있게 한다. 즉, 이 모델은 새로운 작업에 대해 사전 훈련 없이도 단지 무엇을 해야 하는지 문맥 안에서 지시만 받으면 그 작업을 해결할 수 있는 능력을 가진다.

In [19]:
from transformers import pipeline

flan_t5_pipe = pipeline("text2text-generation", model="google/flan-t5-large")

streamer = get_token_generator(flan_t5_pipe)
input_raw_str = "translate English to French: Hello World! How's it going?</s>"

for token in streamer(encoder_input = input_raw_str):
    print(token, end="")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cuda:0


<pad> Bonjour, monde! Comment se passe-t-il?

### Encoder In-Context Learning 

Encoder 방식은 context를 먼저 다 이해한 후 text를 생성한다.

In [ ]:
dataset = [
    {   # Simple Translation
        "premise": "Translate from English to Spanish",
        "question": "The book is on the table.",
        "answer": "El libro está en la mesa.",
        "few_shot": {
            "question": "The cat is sleeping on the sofa.",
            "answer": "El gato está durmiendo en el sofá."
    }},{# Commonsense Reasoning
        "premise": "Answer the question using commonsense knowledge",
        "question": "Why can't a fish live out of water?",
        "few_shot": {
            "question": "Why can't humans breathe underwater?",
            "answer": "Humans can't breathe underwater because they need air, not water, to fill their lungs."
    }},{# Creative Story Generation
        "premise": "Continue the story with a creative twist",
        "question": "Once upon a time, in a forest far away, there was a small bear named Timmy who loved honey.",
        "few_shot": {
            "question": "A princess woke up one day to find her castle floating in the sky.",
            "answer": "As she looked outside, she saw a giant eagle carrying the castle on its back, flying towards the mountains."
    }},{  # Mathematical Problem Solving
        "premise": "Solve the mathematical problem",
        "question": "What is the square root of 144?",
        "few_shot": {
            "question": "What is the cube of 3?",
            "answer": "27."
    }},{# Fact-based Question Answering
        "premise": "Answer based on factual knowledge",
        "question": "Who was the first person to walk on the moon?",
        "few_shot": {
            "question": "Who was the first president of the United States?",
            "answer": "The first president of the United States was George Washington."
    }},{# Conversational Continuation
        "premise": "Continue the conversation naturally",
        "question": "User: Can you help me with directions? Agent:",
        "few_shot": {
            "question": "User: What’s the weather like today? Agent:",
            "answer": "It’s sunny and warm with a light breeze."
    }},{# Conversational Continuation
        "premise": "Continue the conversation naturally",
        "question": (
            "User: Can you help me with directions? Agent: Sure, where to and where from?"
            " User: I'd like to get from LA to San Jose today. What's the best road? Agent: "
        ),
        "few_shot": {
            "question": "User: What’s the weather like today? Agent:",
            "answer": "It’s sunny and warm with a light breeze."
    }}
]

streamer = get_token_generator(flan_t5_pipe)

# encoder 모델을 통해 few-shot
# In-context Learning을 통해 추가 fine-tuning 없이도 문제 유형을 파악해서 답 생성 가능 
for entry in dataset:
    P, Q = entry['premise'], entry['question'] # prefix와 question을 P, Q에 각각 저장 
    FSQ, FSA = entry['few_shot']['question'], entry['few_shot']['answer'] # few_shot의 question과 answer 각각 저장 
    inputs = {
        "encoder_input": f"{P}: {Q}. Example: {FSQ}? {FSA}</s>",
        "decoder_input": "",
        # "encoder_input": f"{P}: ",
        # "decoder_input": f"{FSQ}? {FSA}</s>{Q}? ",
    }
    print(f"{P}: {Q}")
    for token in streamer(**inputs):
        print(token, end="")
    print("\n")

Translate from English to Spanish: The book is on the table.
<pad> El libro está en la mesa.. Example: El gato está dormido en el sofá.? El gato está dormido en 

Answer the question using commonsense knowledge: Why can't a fish live out of water?
<pad> a fish needs water to breathe

Continue the story with a creative twist: Once upon a time, in a forest far away, there was a small bear named Timmy who loved honey.
<pad> The princess was very happy to see her castle floating in the sky.

Solve the mathematical problem: What is the square root of 144?
<pad> -12

Answer based on factual knowledge: Who was the first person to walk on the moon?
<pad> Neil Armstrong

Continue the conversation naturally: User: Can you help me with directions? Agent:
<pad> User: Great. Is there anything else I can do for you?

Continue the conversation naturally: User: Can you help me with directions? Agent: Sure, where to and where from? User: I'd like to get from LA to San Jose today. What's the best road? 

### Flan-T5 Decoder In-Context Learning

GPT나 Ph1 등의 모델은 encoder가 없으며, 한 번에 모든 입력을 받고 그 뒤에 이어지는 다음 토큰을 하니씩 예측한다. 
context는 매 step마다 함께 들어가서 지속적으로 참고한다. 

In [21]:
decoder_pipe = pipeline("text-generation", model="microsoft/phi-1_5", stop_token="\n", device="cuda")
# decoder_pipe = pipeline("text-generation", model="gpt2", stop_token="\n")
streamer = get_token_generator(decoder_pipe)

for entry in dataset:
    P, Q = entry['premise'], entry['question']
    FSP, FSQ = entry['few_shot']['question'], entry['few_shot']['answer']
    inputs = {
        "decoder_input": f"{P}: {FSP}\n\nAnswer: {FSQ}\n\n {P}: {Q}\n\nAnswer: ",
    }
    print("*" * 64)
    for token in streamer(**inputs):
        print(token, end="")
    print("\n")

## EXERCISE: Incorporate Few-Shot (in this case just one-shot) conditioning.

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cuda


****************************************************************
Translate from English to Spanish: The cat is sleeping on the sofa.

Answer: El gato está durmiendo en el sofá.

 Translate from English to Spanish: The book is on the table.

Answer: La libro está en el table.

Exercise 2:

Fill in the blank with the correct word:

The _ is a type of bird that can fly long distances.

Answer: migratory

Ex

****************************************************************
Answer the question using commonsense knowledge: Why can't humans breathe underwater?

Answer: Humans can't breathe underwater because they need air, not water, to fill their lungs.

 Answer the question using commonsense knowledge: Why can't a fish live out of water?

Answer: A fish can't live out of water because it needs water to breathe and survive.


****************************************************************
Continue the story with a creative twist: A princess woke up one day to find her castle floating in the